In [ ]:
# script to Processed the two datasets collected using scrapy and selenium(gdoor.csv and sector.csv)
import pandas as pd
import numpy as np
import re

### Cleaning file containaing company, industry type and sector


In [ ]:
file = pd.DataFrame(pd.read_csv('./Projects/Web_scraping/gdoor.csv'))

In [ ]:
pd.set_option('max_columns',100)
pd.set_option('max_rows',1000)

In [ ]:
f = pd.read_csv('./raw_data/sector.csv', sep='\t', header = None)
f.head()

In [ ]:
f.columns = ['company', 'address','size','year_founded', 'type', 'industry','sector']

In [ ]:
f = f.drop(f.company[f.company.map(lambda x: type(x) is float)].index,0)

f = f.drop(f[f.industry.map(lambda x: type(x) is float)].index,0)

f.sector[f.sector.map(lambda x: type(x) is float)] = 'Unkown'

f.sector[f.sector.str.contains('Unknown')] = f.industry[f.sector.str.contains('Unknown')]

f = f.applymap(lambda x: x.strip())   #or use df.transform(lambda x: x.strip())



### Cleaning file containing list of jobs and respective companies

In [ ]:
f2 = pd.read_csv('./raw_data/gdoor.csv')

In [ ]:
f2= f2.loc[:,['company','days_posted','job_title','location','max_salary','start_salary','rating']]

f2.loc[:,['company','days_posted','job_title']] \
= f2.loc[:,['company','days_posted','job_title']].applymap(lambda x: x.strip())

f2.company = f2.company.map(lambda x: x.strip()) 

In [ ]:
#Extract and normalize posted_days 

def days_posted_extract(x):
    if re.findall('[a-z]+',x)[0] == 'hr':
       val = round(int(re.findall('\d+',x)[0])/24)
    elif re.findall('[a-z]+',x)[0] == 'd':
        val = int(re.findall('\d+',x)[0])
    return val  

In [ ]:
#Extract Salary and normalized

def extract_salary(x):
    if type(x) is float:
        val = 0
    elif type(x) is str:
        val = int(re.findall('\d+',x)[0])
    return val

In [ ]:
f2['days_posted_c'] = f2['days_posted'].map(days_posted_extract)

salary = f2.loc[:,['start_salary', 'max_salary']].applymap(extract_salary)

f2.rename(columns = {'start_salary': 'start_salary_old' , 'max_salary': 'max_salary_old'}, inplace=True)

f2 = pd.concat([f2,salary],axis ='columns')

#convert rating to whole number by rounding 
f2['rating_c'] = f2.rating.map(lambda x: 0 if pd.isnull(x)   else round(x))

#f2 =f2.reindex(f2.columns.tolist() + ['size','year_founded','type','industry','sector'], axis=1) 

In [ ]:
merge_df = pd.merge(f2,f, how='outer', left_on=['company','location'], right_on=['company','address'])

In [ ]:
#extract only columns needed for analysis
merge_df = merge_df.loc[:,['company', 'location','job_title', 'days_posted_c', 'start_salary','max_salary','rating_c',\
                           'address','size','year_founded', 'type', 'industry',\
                           'sector']].drop_duplicates().sort_values('company')

In [ ]:
#Discard data points with invalid address
merge_df.reset_index(inplace=True)
merge_df.drop(['index'],axis=1, inplace=True)
merge_df = merge_df[merge_df.location.isnull() == False]


In [ ]:
#Split address to City and Sate
merge_df['city'] = merge_df.location.map(lambda x: x.split(',')[0] if len(x.split(',')) > 1 else 'Statewide')
merge_df['state']  = merge_df.location.map(lambda x: x.split(',')[1] if len(x.split(',')) > 1 else x)

In [ ]:
#Change State names to Abbrev in some data points
merge_df.state[merge_df.state == 'Oregon'] = 'OR'
merge_df.state[merge_df.state == 'Maryland'] = 'MD'
merge_df.state[merge_df.state == 'Virginia'] = 'VA'
merge_df.state[merge_df.state == 'Nevada'] = 'NV'
merge_df.state[merge_df.state == 'Nebraska'] = 'NE'
merge_df.state[merge_df.state == 'Georgia'] = 'GA'
merge_df.state[merge_df.state == 'Wisconsin'] = 'WI'
merge_df.state[merge_df.state == 'New Jersey'] = 'NJ'
merge_df.state[merge_df.state == 'Illinois'] = 'IL'
merge_df.state[merge_df.state == 'New York State'] = 'NY'
merge_df.state[merge_df.state == 'Utah'] = 'UT'
merge_df.state[merge_df.state == 'Remote'] = 'US'
merge_df.state[merge_df.state == 'United States'] = 'US'

In [ ]:
#Save full processed data
merge_df.to_csv('./clean_data/processed_data.csv', index=False)

In [ ]:
#Save a copy of data with only columns needed for analysis
clean_df = merge_df.loc[:,['company','city','state','year_founded','size','rating_c','job_title','days_posted_c',\
             'start_salary','max_salary']]

In [ ]:
#Save clean data
clean_df.to_csv('./clean_data/clean_data.csv',index=False)